In [65]:
def fitness(x):
    fit = 0
    tam = len(x)
    for i in range(tam):
        for j in range(i + 1, tam):
            vi = x[i]
            vj = x[j]
            c1, c2 = (math.fabs(vi- vj), math.fabs(i - j))
            if c1 == c2:
                fit = fit + 1
    return fit

In [66]:
#Busqueda Tabu

import random
import math

def tabu(time, listaTabu, newTabu):
    updateTime = [(_, time - 1) for (_, time) in listaTabu]
    filterTime = [(_, time) for (_, time) in updateTime if time > 0]
    filterTime.append((newTabu, time))
    return filterTime

def existPair(l, x):
    exist = False
    x1, x2 = x
    for pair in l:
        p1, p2 = pair
        if (p1 == x1 and p2 == x2)  or (p1 == x2 and p2 == x1):
            exist = True
            break
    return exist

def permutar(n, cant):
    combinacionesPosibles = (n*(n+1)) / 2
    if cant > (combinacionesPosibles * 0.5):
        raise("la cantidad de individuos es demasiado alta")
    candidatos = []
    reinas = [x for x in range(n)]
    i = 0
    while i < cant:
        p = random.sample(reinas, 2)
        if not existPair(candidatos, p):
            candidatos.append((p[0], p[1]))
            i = i +1
    return candidatos

def crearIndividuos(indices, base, tabu_list):
    poblacion = []
    for index in indices:
        isTabu = index in [x[0] for x in tabu_list]
        ind  = base[:]
        ind[index[0]], ind[index[1]] = ind[index[1]], ind[index[0]]
        fit = fitness(ind)
        poblacion.append((isTabu, ind, fit, index))
    return poblacion

def nReinasTabu(n, iteraciones, intercambios, time, indInicial):
    if n < 5:
        raise("n debe ser mayor  5")
    if iteraciones < -1 or iteraciones == 0:
        raise("iteraciones debe ser mayor  positivo o -1")
    tabu_list = []
    x = indInicial
    ind = (False, x, fitness(x), (0, 0))
    i = 0
    infinite = False
    if iteraciones is -1 : infinite = True
    while i < iteraciones or infinite:
        #print(i)
        indexCandidatos = permutar(n, intercambios)
        poblacion = crearIndividuos(indexCandidatos, ind[1], tabu_list)
        poblacion.append(ind)
        best = min(poblacion, key=lambda item:item[2])[2]
        l = [a for a in poblacion if a[2] == best and a[0] == False]
        if len(l) == 0:
            l = [a for a in poblacion if a[2] == best and a[0] == True]
        ind = l[0]
        tabu_list = tabu(time, tabu_list, ind[3])
        if ind[2] == 0:
            break
        i = i + 1
    return (ind[1], i)

In [67]:
#Enfriamiento simulado

import random
import math

def crearIndividuo(x, n):
    v = x[:]
    vIndex = random.sample(range(n), 2)
    v[vIndex[0]], v[vIndex[1]] = v[vIndex[1]], v[vIndex[0]]
    return v

def nReinasEnfriamiento(n, extLoop, intLoop, tempInicial, delta, indInicial):
    x = indInicial
    xBest = x
    t = tempInicial
    i = 0
    infinite = False
    if extLoop == -1: infinite = True 
    while i < extLoop or infinite:
        for j in range(intLoop):
            v = crearIndividuo(x, n)
            vfit = fitness(v)
            xfit = fitness(x)
            deltaC = vfit - xfit
            if deltaC < 0:
                x = v[:]
                xBestFit = fitness(xBest)
                if vfit < xBestFit:
                    xBest = v[:]
                    if vfit == 0:
                        infinite = False
                        numIter = i
                        i = extLoop
                        break
            else:
                rnd = random.random()
                umbral = math.exp(-(deltaC / t))
                if rnd < umbral:
                    x = v[:]
        t = t * delta
        i = i + 1
    return (xBest, numIter)

In [68]:
import time
import random
from IPython.display import display, HTML

def crearIndividuoInicial(n):
    return random.sample(range(n), n)

def evaluacionDeAlgoritmos(paramNReinas, paramTabu, paramEnfriamiento):
    #extraer parametros
    n, iteraciones, x = paramNReinas
    inds, timeT = paramTabu
    interLoop, temp, delta = paramEnfriamiento
    
    # ejecutar busqueda tabu
    start = time.time()
    solucionTabu, iterTabu = nReinasTabu(n, iteraciones, inds, timeT, x)
    end = time.time()
    timeTabu = end - start
    
    # ejecutar enfriamiento simulado
    start = time.time()
    solucionEnfriamiento, iterEnfriamiento = nReinasEnfriamiento(n, iteraciones, interLoop, temp, delta, x)
    end = time.time()
    timeEnfriamiento = end - start
    
    return ((solucionTabu, iterTabu, timeTabu, fitness(solucionTabu)), \
            (solucionEnfriamiento, iterEnfriamiento, timeEnfriamiento, fitness(solucionEnfriamiento)))

def resultados(paramExperimento, resultadosTabu, resultadosEnfriamiento):
    solucionTabu, iterTabu, timeTabu, fitTabu = resultadosTabu
    solucionEnfriamiento, iterEnfriamiento, timeEnfriamiento, fitEnfriamiento = resultadosEnfriamiento
    reinas, iteraciones, _ = paramExperimento 
    if iteraciones == -1: 
        itrStr = """<p>Hasta encontrar solución</p>"""  
    else:
        itrStr = """<p>""" + str(iteraciones) + """ iteraciones</p>"""  
    return HTML(""""<h1 style="color:red;" align="center"> Resultados </h1>
        <h2>Realizado con:</h2>
        <p>""" + str(reinas) + """ reinas</p>"""
        + itrStr +
        """<h2>Busqueda tabu </h2>
        <p>Solución: """ + str(solucionTabu) + """ </p>
        <p>Colisiones entre reinas: """+ str(fitTabu) + """ </p>
        <p>Iteraciones utilizadas para hallar la solución: """+ str(iterTabu) + """ </p>
        <p>Tiempo de ejecución: """ + str(timeTabu) + """ sec</p>
        
        <h2>Enfriamiento simulado </h2>
        <p>Solución: """ + str(solucionEnfriamiento) + """ </p>
        <p>Colisiones entre reinas: """+ str(fitEnfriamiento) + """ </p>
        <p>Iteraciones utilizadas para hallar la solución: """+ str(iterEnfriamiento) + """ </p>
        <p>Tiempo de ejecución: """ + str(timeEnfriamiento) + """ sec</p>"""
        )

def realizarExperimento(paramNReinas, paramTabu, paramEnfriamiento):
    resultadosTabu, resultadosEnfriamiento = evaluacionDeAlgoritmos(paramNReinas, paramTabu, paramEnfriamiento)
    return resultados(paramNReinas,resultadosTabu, resultadosEnfriamiento)

In [69]:
"""
Configurar los siguientes parametros para ejecutar el algoritmo

paramNReinas:
    - número de reinas
    - numero de iteraciones (-1 para iterar hasta encontrar solución)
    - individuo inicial (usar método crearIndividuoInicial() para crearlo)
paramTabu:
    - numero de individuos creados por iteración
    - tiempo de tabu
paramEnfiamiento
    - número de iteraciones en el loop interno
    - temperatura inicial
    - tasa de cambio de la temperatura
"""
n =  19 # numero de reinas
x = crearIndividuoInicial(n)
paramNReinas = (n, 1000, x)
paramTabu = (4, 3)
paramEnfriamiento = (10, 100, 0.8)
    
realizarExperimento(paramNReinas, paramTabu, paramEnfriamiento)